In [1]:
#importing libraries:
import requests
import itertools
import pandas as pd
from bs4 import BeautifulSoup
from pandas import DataFrame as df 
import re

In [2]:
# Reading the URL's
URLs = pd.read_csv('CBI_URLs_Final.csv')
URLs_list = pd.DataFrame(URLs)
# print (URL_column)
count_before = len(URLs_list)
print ('The database has', count_before,  'URL listed')

The database has 189 URL listed


In [3]:
URLs_list_cleaned = URLs_list.drop_duplicates(subset=['URLs'])
URLs_list_cleaned.count()
URL_column = URLs_list_cleaned['URLs']
count_after = len(URL_column)
print ('There is', count_after,  'URL listed after removing the duplicates')


There is 183 URL listed after removing the duplicates


In [4]:
# Scraping 1 URL:
# r = requests.get ('https://www.cbi.ca/web/physiotherapy-antigonish-church-strait-area')
# soup = BeautifulSoup (r.text, 'html.parser')
# # print (r.text)
# Gallery = soup.find('div', attrs = {'class' : 'item text-center'})
# # print (Gallery)

In [5]:
   ########################################## Creating all of the Functions ################################################

#Extracting the Location Name 
def Location_Name():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    # print(Locationportlet)
    for i in Locationportlet:
        Location = soup.find('h2').text
    # print (Location)
    return Location


#Extracting the Location Name 
def URL():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    for i in Locationportlet:
        URL = soup.find_all('link')[2]["href"]
        return URL


##Extracting Fax:
def Fax():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        try:
            Fax_start = Location_sentence.find('fax') - 13
            Fax_end = Fax_start + 12
        #     print (Fax_start)
        #     print (Fax_end)
            Fax = Location_sentence[Fax_start:Fax_end]

        except:
            Fax = "Not Informed"
        # print (Fax)
        return Fax
    

##Extracting Phone:
def Phone_number():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Comma_index = Location_sentence.rfind(',')
        # print (Phone_source)
        Phone_start = Comma_index + 21
        Phone_end = Phone_start + 12
        Phone = Location_sentence[Phone_start:Phone_end]
        # print (Phone)
        return Phone

    
##Extracting Postal Code:
def Postal_code():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Comma_index = Location_sentence.rfind(',')
        Postal_code_start = Comma_index + 9
        Postal_code_end = Postal_code_start + 7
        Postal_code = Location_sentence[Postal_code_start:Postal_code_end]
        # print (Postal_code)
        return Postal_code

    
##Extracting Province :
def Province():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
#     print (Locationportlet)

    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Comma_index = Location_sentence.rfind(',')
        Province_start = Comma_index + 2
        Province_end = Province_start + 2
        Province = Location_sentence[Province_start:Province_end]
        # print (Province)
        return Province


##Extracting City:
def City():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
#     print (Locationportlet)

    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Comma_index = Location_sentence.rfind(',')
        sentence_until_comma = Location_sentence[:Comma_index]
        City_start = sentence_until_comma.rfind('br/') + 4
        # print (City_start)
        City = sentence_until_comma[City_start:]
        # print (City)
        return City

    
##Extracting Address_1:
def Address_1():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
#     print (Locationportlet)
    Location_final_index = len(Location_Name())
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Address_1_start = Location_final_index + 8
        Address_1_end = Location_sentence.find('br/',Address_1_start)
        # print (Address_1_start)
        # print (Address_1_end)
        Address_1 = Location_sentence [Address_1_start:Address_1_end]
        # print (Address_1)
        return Address_1

    
##Extracting Address_2:
def Address_2():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
#     print (Locationportlet)
    Location_final_index = len(Location_Name())
    for i in Locationportlet:
        Location_sentence = i.get_text(separator='br/')
        Address_1_start = Location_final_index + 8
        Address_1_end = Location_sentence.find('br/',Address_1_start)
        Address_2_start = Address_1_end + 4
        Address_2_end = Location_sentence.find('br/',Address_2_start)-1
#     print (Address_2_start)
#     print (City_start)
    Comma_index = Location_sentence.rfind(',')
    sentence_until_comma = Location_sentence[:Comma_index]
    City_start = sentence_until_comma.rfind('br/') + 4
    if Address_2_start == City_start:
        Address_2 = '----'
    else:
        Address_2 = Location_sentence [Address_2_start:Address_2_end]
    return Address_2


##Extracting features:
def Features ():
    Features_list = []
    for i in soup.find_all('div', attrs = {'class', 'p-locationfeatures2-feature'}):
        Features_list.append(i.text)
#     print (*Features, sep=", ")
    Feature_1 = ", ".join(Features_list)
#     print (Feature_1)
    return Feature_1


#Extracting Weekdays 
def Weekdays():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    WeekDay_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
#         Hour_open = h.text [ :h.text.find('-')-1]
#         Hour_close = h.text [ h.text.find('-')+2 : ]
        WeekDay_list.append (s.text)
    return WeekDay_list


#Extracting Opening Hours 
def Hour_open():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_open_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_open = h.text [ :h.text.find('-')-1]
#     Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_open_list.append (Hour_open)
    return Hour_open_list


#Extracting closing Hours 
def Hour_close():
    Hours_list = []
    Hoursportlet = soup.find_all('div', attrs = {'class' : 'p-locationfeatures2-main'})
    # print(Hoursportlet)
    Hour_close_list = []
    for s, h in zip(soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col1'}), soup.find_all('td', attrs = {'class' : 'p-locationfeatures2-time-col2'})):
        Hour_close = h.text [ h.text.find('-')+2 : ]
        Hour_close_list.append (Hour_close)        
#     Hour_close = h.text [ h.text.find('-')+2 : ]
    return Hour_close_list


#Extracting Services
def Services():
    Services_source = []
    for i in soup.find_all('div', attrs = {'class', 'row-fluid p-locationservices-bizservices-row'}):
        Services_source.append (i.get_text(separator=';'))

    if len(Services_source) == 2:
        Services_source_1 = Services_source [0] + Services_source [1]
#         print (Services_source_1)
    else:
        Services_source_1 = ''.join(Services_source)

    # print (Services_source_1)

    Services_1 = re.sub('; ;', ';', Services_source_1)
    # print (Services_1)
    Services_2 = re.sub('; ;', ';', Services_1)
    # print (Services_2)
    Services_3 = re.sub(';  ;', ';', Services_2)
    # print (Services_3)
    Services_4 = re.sub('; ;', ';', Services_3)
    # print (Services_4)

    Services = Services_4[Services_4.find(';')+1:Services_4.rfind(';')]
    return Services

#Extracting Side Content
def Side_Content():
    try:
        for i in soup.find_all('div', attrs = {'class' : 'portlet-layout row-fluid'}):
            for p in soup.find_all('div', attrs = {'id' : 'column-6'}):
                Side_Content_Full_Sentence = p.text
            break    

        Side_Content_Full_Sentence_end = Side_Content_Full_Sentence.find('Liferay.Analytics')
        # print (Side_Content_Full_Sentence_end)
        Side_Content = Side_Content_Full_Sentence [7:Side_Content_Full_Sentence_end-4]
        return Side_Content
    except:
        Side_Content = "Not Informed" 
        return Side_Content


# Location_Name(), Address_1(), Address_2(), City(), Province(), Postal_code(), Phone_number(), Fax(), URL(), Features(), Weekdays(), Hour_open(), Hour_close(), Services(), Side_Content()

In [6]:
#Extracting Features - Extracting separeted 
#     Features = []
#     for i in soup.find_all('div', attrs = {'class', 'p-locationfeatures2-feature'}):
#         Features.append(i.text)
#     # print (Features)
#     # print (*Features, sep=", ")
#     Feature_1 = ", ".join(Features)
#     # print (Feature_1)

#     if ' Free Parking ' in Features:
#         Free_Parking = 'Yes'
#     else:
#         Free_Parking = 'No'


#     if ' Wheelchair Accessible ' in Features:
#         Wheelchair_Accessible = 'Yes'
#     else:
#         Wheelchair_Accessible = 'No'


#     if ' Late Hours ' in Features:
#         Late_Hours = 'Yes'
#     else:
#         Late_Hours = 'No'

    
# # print (Free_Parking)
# # print (Wheelchair_Accessible)
# # print (Late_Hours)

In [7]:
 ############################ Loop in URL's and Creating the list to put in a Dataframe #####################################

Final_list = []

 ########################################## Main For Loop on th list of URL's ################################################
for i in URL_column:
    #Reading the URL's (i) on the document:
    r = requests.get (i)
    soup = BeautifulSoup (r.text, 'html.parser')
    # print (r.text)
    General = soup.findAll( 'div' , attrs = {'class' : 'item text-center'})
    # print (General)


 ################################### Loop on the team portlet and creating the list ##########################################

    
    Headshots = soup.find_all('div', attrs = {'class' : 'item text-center'})
#     print (Headshots)
    
    if Headshots == []:
        row = [Location_Name(), 
               Address_1(), 
               Address_2(), 
               Postal_code(), 
               City(), 
               Province(), 
               Phone_number(), 
               Fax(), 
               URL(), 
               Features(),
               Services(),
               Side_Content(),
               'No Team', 
               'No Team', 
               Weekdays(), 
               Hour_open(), 
               Hour_close(),
               'No Team']
        
        Final_list.append(row)
    
    else:    
        for i in Headshots:
#             print (i)
            img = i.find('img', alt=True)
            string1 = img['alt']
            First =  string1.find(',')
            Second = string1.find(',',First+1)
            Name = string1[ : First]
            Job = string1[First+2 : Second]
            Bio = string1[Second+2 : ]
            row = [Location_Name(), 
                   Address_1(), 
                   Address_2(), 
                   Postal_code(), 
                   City(), Province(), 
                   Phone_number(), 
                   Fax(), 
                   URL(), 
                   Features(),
                   Services(),
                   Side_Content(),
                   Name, 
                   Job, 
                   Weekdays(), 
                   Hour_open(), 
                   Hour_close(),
                   Bio]
            
            Final_list.append(row)
            


In [8]:
## Creating the dataframe and ading the list on it 
Dataframe = pd.DataFrame(Final_list)
Dataframe.columns = ['Location', 
                     'Address_1', 
                     'Address_2', 
                     'Postal Code', 
                     'City', 
                     'Province', 
                     'Phone', 
                     'Fax', 
                     'URL', 
                     'Features',
                     'Services',
                     'Description',
                     'Name' , 
                     'Job Title', 
                     'Week_Day', 
                     'Open', 
                     'Close', 
                     'Bio']
# Dataframe.columns = ['Location', 'Day', 'Open', 'Close']
# teste1 = Dataframe.loc[ : ,['Day','Open']]
    
Dataframe

,Location,Address_1,Address_2,Postal Code,City,Province,Phone,Fax,URL,Features,Services,Description,Name,Job Title,Week_Day,Open,Close,Bio
0,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Angele Bonville,Occupational Therapist - Certified Hand Therapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Angele graduated from the University of Albert...
1,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Brandy Deol,Clinic Administrator,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Brandy brings over 18 years of office administ...
2,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Candace Starr,Kinesiologist - Pedorthist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Candace is a registered Kinesiologist and a 20...
3,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Hilda Boer,Physiotherapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Hilda graduated as a Physiotherapist in 1991 i...
4,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Janita le Roux,Occupational Therapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]","Janita grew up in South Africa, and graduated ..."
5,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Jeremy McAllister,Physiotherapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Jeremy has been practicing physiotherapy for o...
6,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Joanne Hillier,Occupational Therapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Joe has practiced Occupational Therapy followi...
7,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair

In [9]:
  ############################################ Downloading the Excel document #################################################
Dataframe.to_csv('CBI_Analysis.csv')

In [13]:
############################################### Reading the Location ID's #####################################################
Loc_ID = pd.read_excel('CBI_Locations_ID.xlsx')
Location_ID = pd.DataFrame(Loc_ID)
Location_ID

,Location,Location_ID
0,CBI Health Centre - Abbotsford,1
1,CBI Health Centre - Abbotsford Clearbrook,2
2,CBI Health Centre - Abbotsford Hand Therapy,3
3,CBI Health Centre - Abbotsford West Railway,4
4,CBI Health Centre - Antigonish,5
5,CBI Health Centre - Barrie Quarry Ridge,6
6,CBI Health Centre - Bedford,7
7,CBI Health Centre - Belleville Millennium,8
8,CBI Health Centre - Berwick,9
9,CBI Health Centre - Brampton County Court,10


In [14]:
Dataframe_with_Loc_ID = Dataframe.merge(Location_ID, left_on='Location', right_on='Location', how='left')
Dataframe_with_Loc_ID ['Location_ID']
# Dataframe_with_Loc_ID.dtypes
# Dataframe_with_Loc_ID ['Location_ID'] = Dataframe_with_Loc_ID['Location_ID'].astype(int)
# Dataframe_with_Loc_ID ['Location_ID'].dtype


Dataframe_with_Loc_ID

,Location,Address_1,Address_2,Postal Code,City,Province,Phone,Fax,URL,Features,Services,Description,Name,Job Title,Week_Day,Open,Close,Bio,Location_ID
0,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Angele Bonville,Occupational Therapist - Certified Hand Therapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Angele graduated from the University of Albert...,1
1,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Brandy Deol,Clinic Administrator,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Brandy brings over 18 years of office administ...,1
2,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Candace Starr,Kinesiologist - Pedorthist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Candace is a registered Kinesiologist and a 20...,1
3,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Hilda Boer,Physiotherapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Hilda graduated as a Physiotherapist in 1991 i...,1
4,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Janita le Roux,Occupational Therapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]","Janita grew up in South Africa, and graduated ...",1
5,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Jeremy McAllister,Physiotherapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Jeremy has been practicing physiotherapy for o...,1
6,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,"Free Parking , Wheelchair Accessible",Acupuncture/IMS;Cognitive Rehabilitation;Custo...,Better health begins here Our CBI Health Cente...,Joanne Hillier,Occupational Therapist,"[Monday, Tuesday, Wednesday, Thursday, Friday]","[7:00am, 7:00am, 7:00am, 7:00am, 7:00am]","[6:00pm, 6:00pm, 6:00pm, 6:00pm, 4:00pm]",Joe has practiced Occupational Therapy followi...,1
7,CBI Health Centre - Abbotsford,34194 Marshall Road,Suite 100,V2S 5E4,Abbotsford,BC,604.852.5655,604.852.5828,https://www.cbi.ca/web/physiotherapy-abbotsfor...,

In [15]:
Dataframe_with_Loc_ID.to_excel('CBI_Location_Page_Analysis.xlsx')